# Regression
## Acquisition and Preparation Exercises
***

##### Let's set up an example scenario as perspective for our regression exercises using the Zillow dataset:

As a Codeup data science graduate, you want to show off your skills to the Zillow data science team in hopes of getting an interview for a position you saw pop up on LinkedIn. _You thought it might look impressive to build an end-to-end project in which you use some of their Kaggle data to predict property values using some of their available features_; who knows, you might even do some feature engineering to blow them away.  
    
         Your goal is to predict the values of single unit properties using the obervations from 2017.
***
_In these exercises, you will complete the first step toward the above goal: acquire and prepare the necessary Zillow data from the zillow database in the Codeup database server_.
***
1. Acquire bedroomcnt, bathroomcnt, calculatedfinishedsquarefeet, taxvaluedollarcnt, yearbuilt, taxamount, and fips from the zillow database for all 'Single Family Residential' properties.
***
2. Using your acquired Zillow data, walk through the summarization and cleaning steps in your wrangle.ipynb file like we did above. You may handle the missing values however you feel is appropriate and meaninful; remember to document your process and decisions using markdown and code commenting where helpful.
***
3. Store all of the necessary functions to automate your process from acquiring the data to returning a cleaned dataframe witn no missing values in your wrangle.py file. Name your final function wrangle_zillow.
***
***

In [9]:
from env import host, username, password, get_db_url
import os
import pandas as pd 
import numpy as np

# 1.) 
- Acquire from the zillow database for all 'Single Family Residential' properties (2017) the following:
   - bedroomcnt
   - bathroomcnt
   - calculatedfinishedsquarefeet
   - taxvaluedollarcn
   - yearbuilt
   - taxamount
   - fips

In [2]:
# query needs to predict the values of single unit properties using observations from 2017
# single family residential is identified at id 261, this must be what is meant by single unit 
# unless 279, inferred single family residential, is relevant as well.
# inferred: deduced or concluded (information) from evidence and reasoning rather than from explicit statements.
# 279 contains a single value, should be fine to avoid. 

# propertylandusetypeid will be the foreign key for joining propertylandusetype

def acquire_zillow_data(use_cache=True):
    if os.path.exists('zillow.csv') and use_cache:
        print('Using cached CSV')
        return pd.read_csv('zillow.csv')
    print('Acquiring data from SQL database')
    df = pd.read_sql('''
                    SELECT bedroomcnt, bathroomcnt, calculatedfinishedsquarefeet,
                    taxvaluedollarcnt, yearbuilt, taxamount, fips
                        FROM properties_2017
                        JOIN propertylandusetype USING(propertylandusetypeid)
                        WHERE propertylandusetypeid = 261
                     '''
                    , get_db_url('zillow'))
    df.to_csv('zillow.csv', index=False)
    
    
    return df

In [63]:
df = acquire_zillow_data()

Using cached CSV


In [4]:
df.head()

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips
0,0.0,0.0,NaN,27516.0,NaN,NaN,6037.0
1,0.0,0.0,NaN,10.0,NaN,NaN,6037.0
2,0.0,0.0,NaN,10.0,NaN,NaN,6037.0
3,0.0,0.0,NaN,2108.0,NaN,174.21,6037.0
4,4.0,2.0,3633.0,296425.0,2005.0,6941.39,6037.0


##### Nailed it. 
                                                      
                                                      moving on.

In [5]:
def describe_data(df):
    print('The first three rows are: ')
    print('----------------------------------------------------------')
    print(df.head(3))
    print('----------------------------------------------------------')
    print("The data frame's shape is: ")
    print('-------------------------')
    print(f' Rows: {df.shape[0]} \n Columns: {df.shape[1]}')
    print('-------------------------')   
    print('The data types and column names are: ')
    print(sorted(df))
    print(df.info())
    print('----------------------------------------------------------')   
    print('The summary statistics are as follows: ')
    print('----------------------------------------------------------')
    print(df.describe())
    print('----------------------------------------------------------')      
    print(f'The number of NA\'s is:')
    print('-------------------------')
    print(df.isna().sum())
    print('-------------------------')
    print ('\nMissing values :  ', df.isnull().sum().values.sum())
    print('----------------------------------------------------------')  
    print('Unique Values for the Columns:')
    print('-------------------------')
    limit = 10
    for col in df.columns:
        if df[col].nunique() < limit:
            print(f'Column: {col} \n')
            print(f'Unique Values: {df[col].unique()} \n')
            print(f'Absolute frequencies: \n {df[col].value_counts()} \n')
            print(f'Relative frequencies: \n {df[col].value_counts(normalize=True)} \n')
        else: 
            print(f'Column: {col} \n')
            print(f'Range of Values: [{df[col].min()} - {df[col].max()}] \n')
        print('-----------------------')
    print('-------Done-zo-------------')

# 2.)
Using your acquired Zillow data, walk through the summarization and cleaning steps in your wrangle.ipynb file like we did above. You may handle the missing values however you feel is appropriate and meaninful; remember to document your process and decisions using markdown and code commenting where helpful.

In [23]:
describe_data(df)

The first three rows are: 
----------------------------------------------------------
   bedroomcnt  bathroomcnt  calculatedfinishedsquarefeet  taxvaluedollarcnt  \
0         0.0          0.0                           NaN            27516.0   
1         0.0          0.0                           NaN               10.0   
2         0.0          0.0                           NaN               10.0   

   yearbuilt  taxamount    fips  
0        NaN        NaN  6037.0  
1        NaN        NaN  6037.0  
2        NaN        NaN  6037.0  
----------------------------------------------------------
The data frame's shape is: 
-------------------------
 Rows: 2152863 
 Columns: 7
-------------------------
The data types and column names are: 
['bathroomcnt', 'bedroomcnt', 'calculatedfinishedsquarefeet', 'fips', 'taxamount', 'taxvaluedollarcnt', 'yearbuilt']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2152863 entries, 0 to 2152862
Data columns (total 7 columns):
 #   Column                

From the describe data function, here's what I immediately glean. 
All of the Dtypes are listed as float64. There are an astonishing 2,152,863 rows among 7 columns.     

The immediately obvious NA rows are in every column except fips, with caluclated finished square feet, yearbuilt, and taxamount having the highest, with 8484, 9337, and 4442 respectively.    

There are a total of 22,778 missing values, which is just about one percent of the full dataframe.

I'll need to more closely look at calculatedfinishedsquarefeet and the range includes 1.0, which is alarming. Same thing with tax amount, since some are only being taxed a dollar. 

****
Planning:
- Detect and remove outliers
- Make an excecutive decision on how to handles the missing values
   - yearbuilt can be filled with 2017. 
- Determine which columns will be most useful, and if any should be dropped
- Potentially create new columns using the available data
- 




In [10]:
def get_object_cols(df):
    '''
    This function takes in a dataframe and identifies the columns that are object types
    and returns a list of those column names. 
    '''
    # create a mask of columns whether they are object type or not
    mask = np.array(df.dtypes == "object")

        
    # get a list of the column names that are objects (from the mask)
    object_cols = df.iloc[:, mask].columns.tolist()
    
    return object_cols

In [12]:
get_object_cols(df)
# as expected, none appeared as everything is listed as float64. 
# The problem, however, is that perhaps some of these are not floats. 

[]

In [16]:
# replace nulls in yearbuilt with 2017
#df['yearbuilt'].fillna(2017, inplace = True)
# checked this using describe_data, no more nulls. 

# after reviewing the spreadsheet, this is a bad idea. 
# in the vast majority of cases where yearbuilt is blank
# calculated finished squarefeet bedroom, bathroom,
# but the taxvaluedollarcnt and taxamount remain. 

In [17]:
describe_data(df)

The first three rows are: 
----------------------------------------------------------
   bedroomcnt  bathroomcnt  calculatedfinishedsquarefeet  taxvaluedollarcnt  \
0         0.0          0.0                           NaN            27516.0   
1         0.0          0.0                           NaN               10.0   
2         0.0          0.0                           NaN               10.0   

   yearbuilt  taxamount    fips  
0     2017.0        NaN  6037.0  
1     2017.0        NaN  6037.0  
2     2017.0        NaN  6037.0  
----------------------------------------------------------
The data frame's shape is: 
-------------------------
 Rows: 2152863 
 Columns: 7
-------------------------
The data types and column names are: 
['bathroomcnt', 'bedroomcnt', 'calculatedfinishedsquarefeet', 'fips', 'taxamount', 'taxvaluedollarcnt', 'yearbuilt']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2152863 entries, 0 to 2152862
Data columns (total 7 columns):
 #   Column                

In [ ]:
 # nifty code from class to check for blanks, might help with nulls

In [19]:
#just in case there are blanks
df = df.replace(r'^\s*$', np.NaN, regex=True)

In [42]:
df.isna().sum()
#didn't help but was worth a shot. 

bedroomcnt                        11
bathroomcnt                       11
calculatedfinishedsquarefeet    8484
taxvaluedollarcnt                493
yearbuilt                       9337
taxamount                       4442
fips                               0
dtype: int64

In [21]:
df.yearbuilt.value_counts()

1955.0    84405
1950.0    71693
1954.0    69163
1953.0    62342
1956.0    61942
          ...  
1823.0        1
1855.0        1
1833.0        1
1840.0        1
1874.0        1
Name: yearbuilt, Length: 154, dtype: int64

In [64]:
df.dropna(axis=0, how='any', inplace=True)

In [59]:
df.isna().sum()

bedroomcnt                      0
bathroomcnt                     0
calculatedfinishedsquarefeet    0
taxvaluedollarcnt               0
yearbuilt                       0
taxamount                       0
fips                            0
dtype: int64

In [47]:
df.shape
# the original number of rows is 2152863
# consequently, 12,628 rows were dropped. 
12628 / 2152863
#only .005 of the dataframe was affected. 

0.005865677472277613

In [69]:
df['fips'] = df.fips.apply(lambda fips: '0' + str(int(fips)))

In [72]:
df.head()
df.dtypes

bedroomcnt                      float64
bathroomcnt                     float64
calculatedfinishedsquarefeet    float64
taxvaluedollarcnt               float64
yearbuilt                       float64
taxamount                       float64
fips                             object
dtype: object

In [61]:
df['fips'] = df['fips'].astype(str)
# as expected, this won't work because it drops the leading 0. 
# it must be a string. 
# str isn't working either

In [66]:
df.head()

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips
4,4.0,2.0,3633.0,296425.0,2005.0,6941.39,6037.0
6,3.0,4.0,1620.0,847770.0,2011.0,10244.94,6037.0
7,3.0,2.0,2077.0,646760.0,1926.0,7924.68,6037.0
11,0.0,0.0,1200.0,5328.0,1972.0,91.60,6037.0
14,0.0,0.0,171.0,6920.0,1973.0,255.17,6037.0


In [73]:
df['yearbuilt'] = df['yearbuilt'].astype(int)

In [74]:
df.dtypes

bedroomcnt                      float64
bathroomcnt                     float64
calculatedfinishedsquarefeet    float64
taxvaluedollarcnt               float64
yearbuilt                         int64
taxamount                       float64
fips                             object
dtype: object

In [37]:
df.head()

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips
4,4.0,2.0,3633.0,296425.0,2005,6941.39,6037.0
6,3.0,4.0,1620.0,847770.0,2011,10244.94,6037.0
7,3.0,2.0,2077.0,646760.0,1926,7924.68,6037.0
11,0.0,0.0,1200.0,5328.0,1972,91.60,6037.0
14,0.0,0.0,171.0,6920.0,1973,255.17,6037.0


In [75]:
#create a new column named 'age', which is 2017 minus the yearbuilt
df['age'] = 2017-df['yearbuilt']

In [76]:
df.head()

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips,age
4,4.0,2.0,3633.0,296425.0,2005,6941.39,06037,12
6,3.0,4.0,1620.0,847770.0,2011,10244.94,06037,6
7,3.0,2.0,2077.0,646760.0,1926,7924.68,06037,91
11,0.0,0.0,1200.0,5328.0,1972,91.60,06037,45
14,0.0,0.0,171.0,6920.0,1973,255.17,06037,44


In [77]:
#rename columns for easier use
df = df.rename(columns={
                        'calculatedfinishedsquarefeet': 'sqft',
                        'bathroomcnt': 'baths',
                        'bedroomcnt': 'beds',
                        'taxvaluedollarcnt':'tax_value'}
              )

In [78]:
df.head()

,beds,baths,sqft,tax_value,yearbuilt,taxamount,fips,age
4,4.0,2.0,3633.0,296425.0,2005,6941.39,06037,12
6,3.0,4.0,1620.0,847770.0,2011,10244.94,06037,6
7,3.0,2.0,2077.0,646760.0,1926,7924.68,06037,91
11,0.0,0.0,1200.0,5328.0,1972,91.60,06037,45
14,0.0,0.0,171.0,6920.0,1973,255.17,06037,44


In [79]:
df2 = acquire_zillow_data()

Using cached CSV


In [80]:
def prepare_zillow(df):
    #just in case there are blanks
    df = df.replace(r'^\s*$', np.NaN, regex=True)

    # drop all nulls, for an affect of .00586 on data
    df.dropna(axis=0, how='any', inplace=True)

    # modify two columns
    df['fips'] = df.fips.apply(lambda fips: '0' + str(int(fips)))
    df['yearbuilt'] = df['yearbuilt'].astype(int)

    #create a new column named 'age', which is the difference of yearbuilt and 2017
    df['age'] = 2017-df['yearbuilt']

    df = df.rename(columns={
                        'calculatedfinishedsquarefeet': 'sqft',
                        'bathroomcnt': 'baths',
                        'bedroomcnt': 'beds',
                        'taxvaluedollarcnt':'tax_value'}
              )

    return df

In [81]:
df2 = prepare_zillow(df2)

In [82]:
df2.head()

,beds,baths,sqft,tax_value,yearbuilt,taxamount,fips,age
4,4.0,2.0,3633.0,296425.0,2005,6941.39,06037,12
6,3.0,4.0,1620.0,847770.0,2011,10244.94,06037,6
7,3.0,2.0,2077.0,646760.0,1926,7924.68,06037,91
11,0.0,0.0,1200.0,5328.0,1972,91.60,06037,45
14,0.0,0.0,171.0,6920.0,1973,255.17,06037,44


In [ ]:
df2.dtypes

In [85]:
describe_data(df2)

The first three rows are: 
----------------------------------------------------------
   beds  baths    sqft  tax_value  yearbuilt  taxamount   fips  age
4   4.0    2.0  3633.0   296425.0       2005    6941.39  06037   12
6   3.0    4.0  1620.0   847770.0       2011   10244.94  06037    6
7   3.0    2.0  2077.0   646760.0       1926    7924.68  06037   91
----------------------------------------------------------
The data frame's shape is: 
-------------------------
 Rows: 2140235 
 Columns: 8
-------------------------
The data types and column names are: 
['age', 'baths', 'beds', 'fips', 'sqft', 'tax_value', 'taxamount', 'yearbuilt']
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2140235 entries, 4 to 2152862
Data columns (total 8 columns):
 #   Column     Dtype  
---  ------     -----  
 0   beds       float64
 1   baths      float64
 2   sqft       float64
 3   tax_value  float64
 4   yearbuilt  int64  
 5   taxamount  float64
 6   fips       object 
 7   age        int64  
dtype

In [ ]:
# Good to go. 
# completed number 3 as well. 